In [2]:
from __future__ import absolute_import, division
from __future__ import print_function, unicode_literals
import pints.toy as toy
import pints
import numpy as np
import logging
import math
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from numpy import inf
import copy 
import pickle
import time
import CMA as CMA

In [11]:
def CMA_on_single_var(opt, 
                 problem,
                 sampler,
                 sampler_x0, 
                 n_chains=1,
                 chain_size=100,
                 need_sensitivities=False, 
                eval_fun=['ESS']):
    
    optimizer_best_fxs = []
    optimizer_best_xs = []
    
    optimizer_best_fx = np.inf
    optimizer_best_x = 0
    
    for _ in range(50):
        
        # Getting the samples of hyper-parameters for the samplers
        optimizer_xs = opt.ask()
        
        # Saving the score of each sample of hyper-parameters
        optimizer_fxs = []
        
        # Evaluate performance for each hyper-parameter configuration
        for x in optimizer_xs:
            
            our_x = [[x[0], x[1], x[2]], x[3], 0, 0]
            
            # Initialise function evaluations and matrix for chains
            function_evaluations = 0 
            chains = []
            
            # Return array of samples for each chain
            for i in range(n_chains):

                # Create sampler object and set hyperparameter
                curr_x0 = sampler_x0[i]
                mcmc = sampler(curr_x0)
                mcmc.set_hyper_parameters(our_x)
                i_chain = []
                
                # Update until we have ``chain_size`` samples
                # Update function evaluations each time we use ask(),tell()
                while len(i_chain) < chain_size:
                    x = mcmc.ask()
                    if need_sensitivities:
                        fx, grad = problem.evaluateS1(x)
                        sample = mcmc.tell((fx, grad))
                    else:
                        fx, _ = problem.evaluateS1(x)
                        sample = mcmc.tell(fx)
                    function_evaluations += 1
                    if sample is not None:
                        i_chain.append(sample)
                
                # Append ith chain to list of chains       
                chains.append(i_chain)
                
            chains = np.array(chains, copy=True)
            optimizer_fx = 0
            # Calculate the score of the sampler with the given hyper-parameters
            # Get the KL if requested
            if 'KL' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                optimizer_fx = kl/len(chains)                
            
            elif 'KL-ITER' in eval_fun:
                kl = 0
                for chain in chains:
                    kl += problem.kl_divergence(chain)
                avg_kl = kl/len(chains)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = avg_kl * avg_iteration_count
            
            # Get the ESS if requested
            elif 'ESS' in eval_fun:
                ess = np.zeros(chains[0].shape[1])
                for chain in chains:
                    ess += np.array(pints._diagnostics.effective_sample_size(chain))
                ess /= len(chains)
                ess = np.min(ess)
                avg_iteration_count = function_evaluations / len(chains)
                optimizer_fx = math.sqrt(avg_iteration_count) / ess
                
            optimizer_fxs.append(optimizer_fx)
        opt.tell(optimizer_fxs)
        
        optimizer_best_fxs.append(opt.fbest())
        optimizer_best_xs.append(opt.xbest())
        
        if opt.fbest() < optimizer_best_fx:
            optimizer_best_fx = opt.fbest()
            optimizer_best_x = opt.xbest()
    
    print(optimizer_best_fx)
    print(optimizer_best_x)
    print()
    
    return optimizer_best_fxs, optimizer_best_xs

In [6]:
rosenbrock = pints.toy.RosenbrockLogPDF()
eggshape = pints.toy.SimpleEggBoxLogPDF(2,4)
Logistic = pints.toy.LogisticModel()

problem = eggshape

# Define the starting points of the samplers
sampler_rosenbrock_x0 = [
    [0,0],
    [3,8],
    [2,4],
]

sampler_egg_x0 = [
    [-10,-10],
    [0,0],
    [10,10],
]

sampler_x0 = sampler_egg_x0

# Define the starting point of the optimzer
dummy = pints.SliceStepoutMCMC([2,4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.expansion_steps()]

boundary = pints.RectangularBoundaries([0,0,1],[20,20,100])

sampler = pints.SliceStepoutMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_single_var(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

0.9148770409186143
[ 5.78142093  5.38034409 49.77944446]

0.9186402995732336
[ 5.03032901  6.013799   67.09158439]

0.9174965939991275
[ 6.52048241  5.46590556 50.81542717]

0.915860251348425
[ 5.15826592  5.25464768 55.50907036]

0.9206269551784219
[ 5.67126491  4.66865795 44.80952176]



In [3]:
dummy = pints.SliceStepoutMCMC([2,4])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.expansion_steps()]
print(optimizer_x0)

[0.2, 0.4, 50]


## Results for the Rosenbrock problem

In [ ]:
# ESS with 50 optimizer iterations
2.3699577396304825
[11.05430603  8.65556335 15.34646559]

2.3540251060136512
[10.68434538  4.13088601 91.90068149]

1.617106711477928
[13.31160578 10.86469062 45.04699239]

2.357389983942719
[ 3.66871274  7.70394962 48.84396455]

2.1588561598210196
[ 9.37848535 14.43081576 85.43210022]

In [ ]:
# ESS with 100 optimizer iterations
2.0388169450864124
[14.4180002   8.32522182 47.13485025]

1.9911451981780695
[ 3.8107873   8.67263218 50.9202088 ]

1.9210559802617702
[ 8.20768945 11.58989919 54.36013239]

1.7975176335682346
[ 9.26843484  9.649202   48.66623769]

1.9065569362301396
[14.63821681  8.11957799 50.73190656]

## Results for the Egg Problem


In [ ]:
0.9148770409186143
[ 5.78142093  5.38034409 49.77944446]

0.9186402995732336
[ 5.03032901  6.013799   67.09158439]

0.9174965939991275
[ 6.52048241  5.46590556 50.81542717]

0.915860251348425
[ 5.15826592  5.25464768 55.50907036]

0.9206269551784219
[ 5.67126491  4.66865795 44.80952176]

## Logistic Problem

In [8]:
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

class ExampleModel(pints.ForwardModel):
    
    def __init__(self):
        self.model = toy.LogisticModel()
    def simulate(self, x, times):
        return self.model.simulate([x[0]/1000, x[1]], times)
    def simulateS1(self, x, times):
        values, gradient = self.model.simulateS1([x[0]/1000, x[1]], times)
        return values, gradient
    def n_parameters(self):
        return 2

# Then create an instance of our new model class
model = ExampleModel()

In [9]:
# Create some toy data
real_parameters = [0.015*10000, 500]
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)
real_parameters = np.array(real_parameters + [noise])

# Get properties of the noise sample
noise_sample_mean = np.mean(values - org_values)
noise_sample_std = np.std(values - org_values)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function (adds an extra parameter!)
log_likelihood = pints.GaussianLogLikelihood(problem)

# Create a uniform prior over both the parameters and the new noise variable
log_prior = pints.UniformLogPrior(
    [0.01*10000, 400, noise * 0.1],
    [0.02*10000, 600, noise * 100],
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
num_chains = 3
sampler_logistic_x0 = [
    real_parameters * 1.01,
    real_parameters * 0.99,
    real_parameters * 1.01
]

In [12]:
problem = log_posterior

sampler_x0 = sampler_logistic_x0

# Define the starting point of the optimzer
dummy = pints.SliceDoublingMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2], dummy.expansion_steps()]

boundary = pints.RectangularBoundaries([0,0,0,1],[20,20,20,15])

sampler = pints.SliceDoublingMCMC

for i in range(5):
    cma = CMA.CMAES(optimizer_x0, boundaries=boundary)
    fxs, xs =  CMA_on_single_var(cma, problem, sampler, 
                        sampler_x0, n_chains=3, chain_size=100, eval_fun=['ESS'])

/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/_log_likelihoods.py:476: RuntimeWarning: invalid value encountered in log
  - np.sum(error**2, axis=0) / (2 * sigma**2))
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/toy/_logistic_model.py:77: RuntimeWarning: overflow encountered in square
  dvalues_dp[:, 0] = k * times * c * exp / (c * exp + 1)**2
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/toy/_logistic_model.py:79: RuntimeWarning: overflow encountered in square
  (self._p0 * (c * exp + 1)**2) + 1 / (c * exp + 1)
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/toy/_logistic_model.py:79: RuntimeWarning: overflow encountered in multiply
  (self._p0 * (c * exp + 1)**2) + 1 / (c * exp + 1)
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/toy/_logistic_model.py:73: RuntimeWarning: overflow encountered in multiply
  values = k / (1 + c * exp)
/home/naunauyoh/anaconda3/lib/python3.7/site-packages/pints/toy/_logistic_model.py:77:

1.6219432789095927
[ 9.01644044  7.34822443 19.32858128 12.03208208]

1.6182562230655584
[ 8.91870117  2.33349609 10.76036024 15.54291153]

1.6095915822776565
[17.61793998  9.04269958  2.91262978 10.1872789 ]

1.5996716791890495
[ 4.54942653  2.8307395   3.23276806 14.28846185]

1.624845442136124
[ 5.52388364  9.47853088  3.30059324 10.02181839]



In [4]:
dummy = pints.SliceDoublingMCMC([2, 4, 10])
optimizer_x0 = [dummy.width()[0], dummy.width()[1], dummy.width()[2], dummy.expansion_steps()]
print(optimizer_x0)

[0.2, 0.4, 1.0, 10]


In [ ]:
1.6219432789095927
[ 9.01644044  7.34822443 19.32858128 12.03208208]

1.6182562230655584
[ 8.91870117  2.33349609 10.76036024 15.54291153]

1.6095915822776565
[17.61793998  9.04269958  2.91262978 10.1872789 ]

1.5996716791890495
[ 4.54942653  2.8307395   3.23276806 14.28846185]

1.624845442136124
[ 5.52388364  9.47853088  3.30059324 10.02181839]